## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node 52ceb7f5-6164-4bf6-b28a-33899f775678

## Importation of libraries

In [2]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *
from navigation.collision_avoidance import *

await node.set_variables(motors(0, 0))

## Main script

In [ ]:

u_old = 0
mu_predict_old = 0
cov_predict_old = 0
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
avoidance_mode = 0
obj_right = False
obstacle_pos = np.array([0, 0])

map = Map()
if P_VISION:
    print(P_SETUP)
    print("Place the Thymio and the destination, check the avoidance_modes and the corners. Then, press S.")

while True:
    map.update(True)
    cv.waitKey(0)
    if cv.waitKey(10) & 0xFF==ord('s'):
        if map.found_corners and map.found_destination:
            map.update()
            break
        else:
            print("USER: Must find corners before start")

if P_VISION: print(P_START)
if P_VISION: map.info()
await node.wait_for_variables()

u_old = np.zeros(2)
mu_predict_old = np.array([map.robot[0], map.robot[1], u_old[0], u_old[1]])
cov_predict_old = 100*np.eye(4)
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
destination = map.destination[0:2]
pos_robot = map.robot[0:2]

#print("mu_predict_old:", mu_predict_old)

while active == True:
    active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    await node.wait_for_variables()
    vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    y = np.array([map.robot[0], map.robot[1], vel_meas[0], vel_meas[1]])
    x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, map.found_robot)
    map.pose_est = np.array([int(x_est[0]), int(x_est[1]), np.arctan2(x_est[3], x_est[2])])
    error_est = np.array([cov_est[0,0], cov_est[1,1], 0])
    print("Pose estimation of the Thymio:", map.pose_est)
    cov_predict_old = cov_est
    mu_predict_old = x_est
    u_old = u
   
    # Check if final destination moved
    change_dest = ((map.destination[1]-destination[1])**2 + (map.destination[0]-destination[0])**2)*0.5
    if change_dest > 20:
        start = True
        print("Nouveau chemin")

    #  Check if the robot has been relocated
    change_robot = ((map.robot[1]-pos_robot[1])**2 + (map.robot[0]-pos_robot[0])**2)*0.5
    if change_robot > 60:
        start = True
        print("Nouveau chemin")
   
    # Global Navigation
    if start == True:
        destination = map.destination[0:2]
        ex_path, corners = compute_visibility_matrix(map.robot[0:2], destination, map.obstacles)
        map.avoidance_modes_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        segment_index = 0
        start = False

    #local nav
    u, avoidance_mode, segment_index, obj_right, obstacle_pos = await collision_avoidance(path, node, map.robot, avoidance_mode, segment_index, obj_right, obstacle_pos) # Zacharie
    #print("avoidance_mode", avoidance_mode)

    if not avoidance_mode:
        u, segment_index, end = path_direction(map.robot, path, segment_index)
    
    #update robot position
    pos_robot = map.robot[0:2]

    await node.set_variables(motors(int(u[0]), int(u[1])))

    if cv.waitKey(20) & 0xFF==ord('p'):
        map.vision_stop()
        break

Center button pressed. Stopping the algorithm.


## Thymio velocity measurements

In [ ]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    await node.set_variables(motors(100, 100))
    vel_meas.append(velocity_with_odometry(node, 0)[0])
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[10:])
vel_meas_var = np.var(vel_meas[10:])
print("Length of vel_meas:", len(vel_meas))
print("Velocity measured:", vel_meas[10:])
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

Center button pressed. Stopping the algorithm.
Length of vel_meas: 1
Velocity measured: []
Velocity mean: nan
Velocity variance: nan


c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.

In [5]:
import time

timer = 0
start = time.time()

while timer < 10:
    await node.set_variables(motors(97, 100))
    timer = time.time() - start

await node.set_variables(motors(0, 0))

{'error_code': 2}

In [6]:
await node.set_variables(motors(0, 0))


{'error_code': 2}

In [7]:
robot_vel = np.array([3.35,3.36,3.35,3.39,3.36,3.33,3.34,3.38,3.42,3.41])

robot_vel_mean = np.mean(robot_vel)
robot_vel_var = np.var(robot_vel)

print("Robot velocity mean:", robot_vel_mean)
print("Robot velocity variance:", robot_vel_var)

Robot velocity mean: 3.3689999999999998
Robot velocity variance: 0.0008090000000000007


## Thymio position measurements

In [8]:
map = Map()

robot_positions_x = []
robot_positions_y = []
map.update(True)
while True:
    map.update()
    robot_position_x = map.robot[0]
    robot_position_y = map.robot[1]
    robot_positions_x.append(robot_position_x)
    robot_positions_y.append(robot_position_y)
    if cv.waitKey(20) & 0xFF==ord("d"):
        map.vision_stop()
        break

robot_positions_x_mean = np.mean(robot_positions_x[10:])
robot_positions_x_var = np.var(robot_positions_x[10:])
robot_positions_y_mean = np.mean(robot_positions_y[10:])
robot_positions_y_var = np.var(robot_positions_y[10:])

print("Position x measured:", robot_positions_x[10:])
print("Position x mean:", robot_positions_x_mean)
print("Position x variance:", robot_positions_x_var)

print("Position y measured:", robot_positions_y[10:])
print("Position y mean:", robot_positions_y_mean)
print("Position y variance:", robot_positions_y_var)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-27410],
 '_imot': [-17, -189],
 '_integrator': [689, -1116],
 '_productId': [8],
 '_vbat': [718, 717],
 'acc': [1, -1, 24],
 'acc._tap': [32],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10655, 9142, 13730, 13675, 15078],
 'buttons._noise': [57, 46, 60, 53, 80],
 'buttons._raw': [10641, 9140, 13538, 13663, 15020],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-27410],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [261],
 'mic.intensity': [29],
 'mic.threshold': [0],
 'motor.left.pwm': [329],
 'motor.left.speed': [-72],
 'motor.left.target': [-100],
 'motor.right.pwm': [-419],
 'motor.right.speed': [100],
 'motor.right.targ